In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import optuna
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import sklearn

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

/home/nlztrk/anaconda3/envs/wildfire/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


We are reading the train, validation and test splits.

In [2]:
train = pd.read_csv("../data/train_val_test_data/train.csv")
valid = pd.read_csv("../data/train_val_test_data/valid.csv")
test = pd.read_csv("../data/train_val_test_data/test.csv")

We are setting our target feature columns.

In [3]:
features = [
    'latitude', 'longitude', 'month',
    'fire_cnt_before', 'fire_before',
    'fire_cnt_last_year', 'fire_last_year',
    'fire_cnt_last_year_same_month', 'fire_last_year_same_month',
    'temperature_min', 'temperature_max', 'temperature_avg'
]

We are defining our objective function to optimize. We want to maximize our ROC-AUC score for our classifier.

In [4]:
def objective(trial):

    train_data = lgb.Dataset(train[features], label=train.fire)
    
    param = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02,0.05]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,50,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('cat_smooth', 1, 256),
        'cat_l2' : trial.suggest_int('cat_smooth', 1, 256),
    }
    

    gbm = lgb.train(param, train_data)
    preds = gbm.predict(valid[features])
    pred_labels = np.rint(preds)
    accuracy = roc_auc_score(valid.fire, pred_labels)
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, n_jobs=-1)

[I 2022-02-03 20:24:34,723] A new study created in memory with name: no-name-6e6b9f23-1de9-49e5-9ab8-b0ed2247b92f
/home/nlztrk/anaconda3/envs/wildfire/lib/python3.8/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(


[LightGBM] [Warning] bagging_fraction is set=0.9894322719695299, subsample=0.6 will be ignored. Current value: bagging_fraction=0.9894322719695299
[LightGBM] [Warning] feature_fraction is set=0.9712647703472412, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.9712647703472412
[LightGBM] [Warning] bagging_fraction is set=0.13722464247278265, subsample=0.7 will be ignored. Current value: bagging_fraction=0.13722464247278265
[LightGBM] [Warning] feature_fraction is set=0.2271504889281947, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.2271504889281947
[LightGBM] [Warning] bagging_fraction is set=0.14910949063451812, subsample=1.0 will be ignored. Current value: bagging_fraction=0.14910949063451812
[LightGBM] [Warning] feature_fraction is set=0.44683745683720355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.44683745683720355
[LightGBM] [Warning] bagging_fraction is set=0.4330114343916659, subsample=0.4 will be igno

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] bagging_fraction is set=0.5640517826144018, subsample=0.4 will be ignored. Current value: bagging_fraction=0.5640517826144018
[LightGBM] [Warning] feature_fraction is set=0.3720368800229398, colsample_bytree=0.3 will be ignored. 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[I 2022-02-03 20:27:14,100] Trial 94 finished with value: 0.6905350075149167 and parameters: {'lambda_l1': 5.202091033850344e-06, 'lambda_l2': 1.2096944769271418e-05, 'colsample_bytree': 0.7, 'subsample': 0.5, 'learning_rate': 0.05, 'max_depth': 50, 'num_leaves': 661, 'feature_fraction': 0.8757364448269976, 'bagging_fraction': 0.7463002608130608, 'bagging_freq': 11, 'min_child_samples': 173, 'cat_smooth': 236}. Best is trial 94 with value: 0.6905350075149167.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2022-02-03 20:27:14,208] Trial 91 finished with value: 0.6545227437001347 and parameters: {'lambda_l1': 0.7983313151292082, 'lambda_l2': 3.495692344159969, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.05, 'max_depth': 50, 'num_leaves': 830, 'feature_fraction': 0.9640678138310316, 'bagging_fraction': 0.929162474053431, 'bagging_freq': 10, 'min_child_samples': 10, 'cat_smooth': 192}. Best is trial 94 with value: 0.6905350075149167.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2022-02-03 20:27:18,304] Trial 90 finished with value: 0.5965764201726491 and parameters: {'lambda_l1': 0.9086110013818623, 'lambda_l2': 0.5190168629708157, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.05, 'max_depth': 100, 'num_leaves': 837, 'feature_fraction': 0.49786039166529866, 'bagging_fraction': 0.9306508325837282, 'bagging_freq': 1, 'min_child_samples': 13, 'cat_smooth': 191}. Best is trial 94 with value: 0.6905350075149167.
[I 2022-02-03 20:27:20,016] Trial 99 finished with value: 0.6876844669463875 and parameters: {'lambda_l1': 1.118371338446522e-06, 'lambda_l2': 1.5524658904421668e-05, 'colsample_bytree': 0.6, 'subsample': 0.5, 'learning_rate': 0.05, 'max_depth': 50, 'num_leaves': 558, 'feature_fraction': 0.8795218972753192, 'bagging_fraction': 0.7549847468722802, 'bagging_freq': 11, 'min_child_samples': 182, 'cat_smooth': 234}. Best is trial 94 with value: 0.6905350075149167.
[I 2022-02-03 20:27:20,402] Trial 60 finished with value: 0.5021027312963618 an

We are taking a look to our best fitted parameters. We are going to use these in the final training of our model.

In [5]:
study.best_params

{'lambda_l1': 5.202091033850344e-06,
 'lambda_l2': 1.2096944769271418e-05,
 'colsample_bytree': 0.7,
 'subsample': 0.5,
 'learning_rate': 0.05,
 'max_depth': 50,
 'num_leaves': 661,
 'feature_fraction': 0.8757364448269976,
 'bagging_fraction': 0.7463002608130608,
 'bagging_freq': 11,
 'min_child_samples': 173,
 'cat_smooth': 236}